## Eventos INM a Nivel municipio

Notebook para trabajar datos de encuentros con migrantes a nivel municipal.
El objetivo de este notebook es extraer dos documentos;

1)Archivo Geoespacial con los boundaries municipales para implementar en arcgis

2)Archivo csv + anexos geoespaciales para generar mapas con proyecciones estadisticas

### Eventos de extranjeros presentados ante la autoridad migratoria, según municipio.

Procesamos las tablas estadisticas con datos sobre eventos de migrantes irregulares registrados por el INM por entidad federativa.

Los datos seleccionados se refieren unicamente a los eventos en los que migrantes fueron  ...

    2001 - 2012 la información incluye eventos de extranjeros en situación migratoria irregular a los que el Instituto Nacional de Migración (INM) les inició un procedimiento administrativo de aseguramiento y a los centroamericanos acogidos al “Memorándum de entendimiento entre los gobiernos de los Estados Unidos Mexicanos, de la República de El Salvador, de la República de Guatemala, de la República de Honduras y de la República de Nicaragua, para la repatriación digna, ordenada, ágil y segura de nacionales centroamericanos migrantes vía terrestre” suscrito el 5 de mayo de 2006 y su anexo del 26 de abril de 2007.

    2013 - 2020 la información incluye los eventos de migrantes en situación migratoria irregular a los que se les inició un procedimiento administrativo de presentación por no acreditar su situación migratoria, según lo previsto en los arts. 99, 112 y 113 de la Ley de Migración y del art. 222 de su Reglamento.

    2021 - 2023 la información hace referencia a los eventos de extranjeros en situación migratoria irregular a los que se les inicio un Procedimiento Administrativo Migratorio (PAM) ante el Instituto Nacional de Migración (INM) por no acreditar su situación migratoria, según lo previsto en los arts. 99, 100, 101 y 113 de la Ley de Migración y del art. 222 de su Reglamento 

Y a los que fueron canalizados por esta misma autoridad a los albergues de la red DIF, con PAM iniciado; según lo previsto en los art. 112 y 113 de la Ley de Migración y del art. 222 de su Reglamento, así como de los arts. 89, 94 de la Ley General de los Derechos de Niñas, Niños y Adolescentes.(Excluidos en estas BD para mantener comparabilidad estadistica)

El nombre de datos en el tiempo:

2002 - 2006 "EVENTOS DE ASEGURAMIENTO EN MÉXICO SEGÚN DELEGACIÓN REGIONAL"

2007 - 2012  "Eventos de extranjeros alojados en estaciones migratorias, según entidad federativa"

2013 - 2023 "Eventos de extranjeros presentados ante la autoridad migratoria, según entidad federativa"



# Introducción
Este notebook analiza la dinámica de flujos migratorios irregulares en México, utilizando herramientas de análisis espacial como GeoPandas.

Fuente: http://www.politicamigratoria.gob.mx/es/PoliticaMigratoria/Boletines_Estadisticos

## Configuración Inicial e Importaciones

In [1]:
#Librerias Necesarias
import pandas as pd
import glob
import numpy as np
import seaborn as sns
from unidecode import unidecode
import re
import os
import sys
import geopandas as gpd
from geodatasets import get_path


from geopy.geocoders import Photon
from geopy.exc import GeocoderTimedOut
from shapely.geometry import Point


import matplotlib.pyplot as plt
import squarify
import seaborn as sns
import matplotlib as mpl

import plotly.express as px
import plotly.graph_objects as go

## Preparación de los Datos

### Carga de Datos INM Municipal (Archivo no geoespacial)

La limpieza y prepraración de set de datos Incluye:

    -1 revisar el archivo desde excel para remover todas las columnas que corresponden a titulos y notas.
    -2 cargar la bd en python
    -3 deberemos normalizar las columnas para que correspondan a un mes.
    en el caso de los datos posteriores al año 2023 los cuales tienen columnas a dos niveles: mes (presentados, canalizados y subtotal), deberemos combinar las columnas con cada mes: "enero_presentados", "enero_canalizados", "enero_subtotal"

In [2]:
# Ruta al archivo .xls
archivo_xls = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/2. datos_estadisticos/1. Detenciones INM/Total_De_Eventos/Eventos Municipal/3.1_mun_2023.xlsx'

# Cargar el archivo xls como un DataFrame de pandas
datos = pd.read_excel(archivo_xls)

Eliminar columnas y filas innecesarias: Podemos quitar las columnas y filas que solo contienen valores NaN o que no aportan información útil para el análisis.

In [3]:
# Eliminar las columnas completamente vacías
datos = datos.dropna(axis=1, how='all')

# Eliminar las filas completamente vacías
datos = datos.dropna(axis=0, how='all')

Renombrar las columnas: Las columnas deben tener nombres descriptivos para facilitar el entendimiento y el acceso a los datos.

In [4]:
# Establecer la primera fila como nombres de columna
datos.columns = datos.iloc[0]

# Descartar la primera fila del DataFrame
datos = datos.drop(datos.index[0])

# Resetea el índice si es necesario
datos = datos.reset_index(drop=True)

In [5]:
#lista de columnas:
lista_columnas = datos.columns.tolist()
print(lista_columnas)

['Entidad Federativa / Municipio', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Subtotal', 'Presentados', 'Canalizados', 'Total']


Sabemos del df original que cada set de 3 columans corresponde a 1 mes despues de la columna [0], por lo que es mas facil renombrarlas:

In [6]:
# Nuevo mapeo de nombres para las columnas
nuevos_nombres = [
    'Entidad Federativa / Municipio',
    'enero_presentados', 'enero_canalizados', 'enero_total',
    'febrero_presentados', 'febrero_canalizados', 'febrero_total',
    'marzo_presentados', 'marzo_canalizados', 'marzo_total',
    'abril_presentados', 'abril_canalizados', 'abril_total',
    'mayo_presentados', 'mayo_canalizados', 'mayo_total',
    'junio_presentados', 'junio_canalizados', 'junio_total',
    'julio_presentados', 'julio_canalizados', 'julio_total',
    'agosto_presentados', 'agosto_canalizados', 'agosto_total',
    'septiembre_presentados', 'septiembre_canalizados', 'septiembre_total',
    'octubre_presentados', 'octubre_canalizados', 'octubre_total',
    'noviembre_presentados', 'noviembre_canalizados', 'noviembre_total',
    'diciembre_presentados', 'diciembre_canalizados', 'diciembre_total',
    'total_presentados', 'total_canalizados', 'total_total'
]

# Asignación de nuevos nombres a las columnas del DataFrame
datos.columns = nuevos_nombres




In [13]:
datos.head()

,Entidad Federativa / Municipio,tipo,enero_presentados,enero_canalizados,enero_total,febrero_presentados,febrero_canalizados,febrero_total,marzo_presentados,marzo_canalizados,...,noviembre_presentados,noviembre_canalizados,noviembre_total,diciembre_presentados,diciembre_canalizados,diciembre_total,total_presentados,total_canalizados,total_total,ciudad
0,Total general,estado,24801,12559,37360,23631,14410,38041,31360,13268,...,72789,25180,97969,79946,15507,95453,566361,215815,782176,Total general
1,Aguascalientes,estado,153,13,166,155,19,174,17,13,...,3,25,28,0,16,16,386,173,559,Aguascalientes
2,"Aguascalientes, Ags.",municipio,153,13,166,155,19,174,15,12,...,0,0,0,0,0,0,348,75,423,"Aguascalientes, Aguascalientes"
3,"Cosío, Ags.",municipio,0,0,0,0,0,0,2,1,...,1,20,21,0,16,16,27,71,98,"Cosío, Aguascalientes"
4,"Jesús María, Ags.",municipio,0,0,0,0,0,0,0,0,...,2,5,7,0,0,0,8,12,20,"Jesús María, Aguascalientes"


creamos nueva columna ciudad, la cual contiene valores: "municipio, estado"

In [10]:
# Crea la columna 'tipo' basada en la presencia de una coma en 'Entidad Federativa / Municipio'
datos['tipo'] = datos['Entidad Federativa / Municipio'].apply(lambda x: 'municipio' if ',' in x else 'estado')

# Mueve la columna 'tipo' a la posición deseada, que es la segunda posición (índice 1)
# Primero, guardamos la columna 'tipo' y luego la eliminamos de su posición original
columna_tipo = datos['tipo']
datos.drop(columns=['tipo'], inplace=True)

# Luego, insertamos la columna en la posición deseada
datos.insert(1, 'tipo', columna_tipo)

In [12]:
# Primero, inicializa la columna 'ciudad' con valores vacíos.
datos['ciudad'] = ''

# Almacenar el estado actual mientras iteramos.
estado_actual = None

# Iterar sobre las filas del dataframe.
for index, row in datos.iterrows():
    if row['tipo'] == 'estado':
        # Cuando encuentra un 'estado', actualiza el estado actual y asigna el nombre del estado a la columna 'ciudad'.
        estado_actual = row['Entidad Federativa / Municipio']
        datos.at[index, 'ciudad'] = estado_actual
    elif row['tipo'] == 'municipio' and estado_actual:
        # Para los 'municipio', combina el nombre del municipio con el estado actual.
        ciudad = row['Entidad Federativa / Municipio'].split(',')[0]  # Asumiendo que el nombre de la ciudad está antes de la coma.
        datos.at[index, 'ciudad'] = f"{ciudad}, {estado_actual}"


Hacemos un subdf con los municipios, y dropeamos las columnas que ya no necesitamos

In [14]:
# Filtrado para obtener solo las filas donde 'tipo' es igual a 'municipio'
datos_municipio = datos.loc[datos['tipo'] == 'municipio']

In [16]:
# Mover la columna 'ciudad' al principio del dataframe
col_order = ['ciudad'] + [col for col in datos_municipio.columns if col not in ['ciudad', 'Entidad Federativa / Municipio', 'tipo']]
datos_municipio = datos_municipio[col_order]


In [18]:
datos_municipio.shape

(389, 40)

In [31]:
datos_municipio.head()

,ciudad,Latitude,Longitude,Coordinates,enero_presentados,enero_canalizados,enero_total,febrero_presentados,febrero_canalizados,febrero_total,...,octubre_total,noviembre_presentados,noviembre_canalizados,noviembre_total,diciembre_presentados,diciembre_canalizados,diciembre_total,total_presentados,total_canalizados,total_total
2,"Aguascalientes, Aguascalientes",22.000000,-102.500000,"22.0000001,-102.5000001",153,13,166,155,19,174,...,30,0,0,0,0,0,0,348,75,423
3,"Cosío, Aguascalientes",22.370515,-102.312009,"22.3705155,-102.31200851052341",0,0,0,0,0,0,...,27,1,20,21,0,16,16,27,71,98
4,"Jesús María, Aguascalientes",21.934556,-102.468452,"21.934555500000002,-102.46845171651151",0,0,0,0,0,0,...,0,2,5,7,0,0,0,8,12,20
5,"San Francisco de los Romo, Aguascalientes",22.074628,-102.270713,"22.0746282,-102.2707133",0,0,0,0,0,0,...,3,0,0,0,0,0,0,3,15,18
7,"Ensenada, Baja California",31.865889,-116.602983,"31.8658887,-116.602983",4,0,4,1,0,1,...,0,0,0,0,1,0,1,23,6,29


Agregamos una columna geoespacial para hacer la union con la df de los poligonos municipales.

In [19]:
# Inicializar el geolocalizador de Photon
geolocator = Photon(user_agent="geoapiExercises")

# Función para obtener latitud, longitud y coordenadas combinadas
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude, f"{location.latitude},{location.longitude}")
        else:
            return (None, None, None)
    except:
        return (None, None, None)

# Aplicar la función a la columna 'Entidad Federativa / Municipio'
datos_municipio['Lat_Long_Coords'] = datos_municipio['ciudad'].apply(get_lat_long)

# Separar las coordenadas en tres columnas nuevas: 'Latitude', 'Longitude' y 'Coordinates'
datos_municipio[['Latitude', 'Longitude', 'Coordinates']] = pd.DataFrame(datos_municipio['Lat_Long_Coords'].tolist(), index=datos_municipio.index)

# Eliminar la columna 'Lat_Long_Coords' ya que se ha dividido en otras columnas
datos_municipio.drop(columns=['Lat_Long_Coords'], inplace=True)

In [25]:
last_three_cols = datos_municipio.columns[-3:].tolist()

# Define el nuevo orden de las columnas: 'ciudad', seguido por las últimas tres columnas, y luego todas las demás, excepto las últimas tres.
new_col_order = ['ciudad'] + last_three_cols + [col for col in datos_municipio.columns if col not in ['ciudad'] + last_three_cols]

# Reordena las columnas usando el nuevo orden
datos_municipio = datos_municipio[new_col_order]

Preparar una columna geometry para unir posteriormente

In [37]:
# Convertir el DataFrame de coordenadas a un GeoDataFrame si aún no lo es
gdf_datos_municipio = gpd.GeoDataFrame(
    datos_municipio,
    geometry=gpd.points_from_xy(datos_municipio.Longitude, datos_municipio.Latitude)
)

In [38]:
gdf_datos_municipio

,ciudad,Latitude,Longitude,Coordinates,enero_presentados,enero_canalizados,enero_total,febrero_presentados,febrero_canalizados,febrero_total,...,noviembre_presentados,noviembre_canalizados,noviembre_total,diciembre_presentados,diciembre_canalizados,diciembre_total,total_presentados,total_canalizados,total_total,geometry
2,"Aguascalientes, Aguascalientes",22.000000,-102.500000,"22.0000001,-102.5000001",153,13,166,155,19,174,...,0,0,0,0,0,0,348,75,423,POINT (-102.50000 22.00000)
3,"Cosío, Aguascalientes",22.370515,-102.312009,"22.3705155,-102.31200851052341",0,0,0,0,0,0,...,1,20,21,0,16,16,27,71,98,POINT (-102.31201 22.37052)
4,"Jesús María, Aguascalientes",21.934556,-102.468452,"21.934555500000002,-102.46845171651151",0,0,0,0,0,0,...,2,5,7,0,0,0,8,12,20,POINT (-102.46845 21.93456)
5,"San Francisco de los Romo, Aguascalientes",22.074628,-102.270713,"22.0746282,-102.2707133",0,0,0,0,0,0,...,0,0,0,0,0,0,3,15,18,POINT (-102.27071 22.07463)
7,"Ensenada, Baja California",31.865889,-116.602983,"31.8658887,-116.602983",4,0,4,1,0,1,...,0,0,0,1,0,1,23,6,29,POINT (-116.60298 31.86589)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,"Valladolid, Yucatán",20.690251,-88.201700,"20.6902505,-88.2016999",0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,5,POINT (-88.20170 20.69025)
418,"Calera, Zacatecas",22.978480,-102.856223,"22.978479999999998,-102.85622288611614",0,0,0,0,0,0,...,1,11,12,0,0,0,26,16,42,POINT (-102.85622 22.97848)
419,"Jerez, Zacatecas",22.710593,-103.001473,"22.7105933,-103.00147264243049",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,POINT (-103.00147 22.71059)
420,"Trancoso, Zacatecas",22.735302,-102.366797,"22.7353025,-102.3667969",0,0,0,0,0,0,...,1,2,3,0,0,0,17,79,96,POINT (-102.36680 22.73530)


### Geolocalizaciones: Instituciones

##### Cargado Limites Municipales de Mexico

In [22]:
# Mex Municipios
mx_mun = gpd.read_file('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/2. arcgis/maps_boundaries/Municipalities/mapa_mexico_mun/').set_index('CLAVE').to_crs(epsg=4485)

In [23]:
mx_mun.shape

(2480, 6)

Trabajaremos aada municipio uniendolo con la columna clave

In [47]:
mx_mun.head()

,NOM_MUN,NOMEDO,CVE_EDO,CVE_MUNI,Area,geometry
CLAVE,,,,,,
02004,Tijuana,Baja California,02,004,1122.661145,"POLYGON ((-73565.018 3602427.487, -73564.403 3..."
02003,Tecate,Baja California,02,003,3670.991923,"POLYGON ((-38995.078 3617846.589, -31557.921 3..."
02002,Mexicali,Baja California,02,002,13119.275713,"POLYGON ((48160.716 3621731.593, 58570.990 362..."
02005,Playas de Rosarito,Baja California,02,005,517.120801,"POLYGON ((-70946.724 3594803.753, -70966.034 3..."
26055,San Luis Rio Colorado,Sonora,26,055,9033.770278,"POLYGON ((127160.493 3587762.823, 127099.688 3..."


In [33]:
print(mx_mun.iloc[0, mx_mun.columns.get_loc('geometry')])


POLYGON ((-73565.01806153602 3602427.4871878405, -73564.40301941778 3602430.261461328, -73524.09259320144 3602470.127627468, -73498.21775738243 3602480.729576586, -73501.24511785258 3602502.166990318, -73577.09446649416 3602494.4001495442, -73615.57125824143 3602559.0446542674, -73610.81429180468 3602642.985894397, -73623.47473697248 3602723.3442597575, -73670.97894105816 3602772.084386826, -73692.8128856424 3602831.790627385, -73669.38862416334 3602872.5902766516, -73683.01707404864 3602914.7261859584, -73675.11887167988 3602981.351914688, -73636.75041561865 3603004.531123287, -73625.47083946271 3603046.2433388657, -73654.57135187706 3603063.8063031957, -73713.10899624263 3603059.2436413965, -73748.90549239749 3603086.3403284904, -73734.12772912893 3603131.8386828853, -73690.98299110157 3603150.3819613373, -73678.18781139364 3603233.211558745, -73719.43391625548 3603323.0475804615, -73710.00512772868 3603418.814162628, -73742.03968630661 3603437.7331321794, -73799.73100376886 3603449.

### Union de ambas bd.

-utilizando los datos geoespaciales para hacer el merge

In [42]:
# Asegúrate de que gdf_datos_municipio y mx_mun tengan el mismo CRS
gdf_datos_municipio = gdf_datos_municipio.set_crs(mx_mun.crs, allow_override=True)


In [49]:
# Realizar la unión espacial
union_geoespacial = gpd.sjoin(gdf_datos_municipio, mx_mun, how="left", op='intersects')


/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [52]:
union_geoespacial.tail()

,ciudad,Latitude,Longitude,Coordinates,enero_presentados,enero_canalizados,enero_total,febrero_presentados,febrero_canalizados,febrero_total,...,total_presentados,total_canalizados,total_total,geometry,index_right,NOM_MUN,NOMEDO,CVE_EDO,CVE_MUNI,Area
416,"Valladolid, Yucatán",20.690251,-88.201700,"20.6902505,-88.2016999",0,0,0,0,0,0,...,5,0,5,POINT (-88.202 20.690),NaN,NaN,NaN,NaN,NaN,NaN
418,"Calera, Zacatecas",22.978480,-102.856223,"22.978479999999998,-102.85622288611614",0,0,0,0,0,0,...,26,16,42,POINT (-102.856 22.978),NaN,NaN,NaN,NaN,NaN,NaN
419,"Jerez, Zacatecas",22.710593,-103.001473,"22.7105933,-103.00147264243049",0,0,0,0,0,0,...,1,0,1,POINT (-103.001 22.711),NaN,NaN,NaN,NaN,NaN,NaN
420,"Trancoso, Zacatecas",22.735302,-102.366797,"22.7353025,-102.3667969",0,0,0,0,0,0,...,17,79,96,POINT (-102.367 22.735),NaN,NaN,NaN,NaN,NaN,NaN
421,"Zacatecas, Zacatecas",23.082358,-103.208570,"23.0823582,-103.20856984509811",216,45,261,66,9,75,...,795,191,986,POINT (-103.209 23.082),NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# Asegurar que ambos GeoDataFrames usen el mismo CRS
if gdf_datos_municipio.crs != mx_mun.crs:
    gdf_datos_municipio.to_crs(mx_mun.crs, inplace=True)

# Intentar nuevamente la unión espacial
union_geoespacial = gpd.sjoin(gdf_datos_municipio, mx_mun, how="left", op='intersects')

# Verificar si la columna 'index_right' está presente
if 'index_right' in union_geoespacial.columns:
    print("La unión espacial ha sido exitosa y 'index_right' está presente.")
else:
    print("La unión espacial falló. 'index_right' no está presente. Verifica los datos y los CRS.")


La unión espacial ha sido exitosa y 'index_right' está presente.


/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [57]:
union_geoespacial

,ciudad,Latitude,Longitude,Coordinates,enero_presentados,enero_canalizados,enero_total,febrero_presentados,febrero_canalizados,febrero_total,...,total_canalizados,total_total,geometry_points,index_right,NOM_MUN,NOMEDO,CVE_EDO,CVE_MUNI,Area,index_mx_mun
2,"Aguascalientes, Aguascalientes",22.000000,-102.500000,"22.0000001,-102.5000001",153,13,166,155,19,174,...,75,423,POINT (-102.500 22.000),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Cosío, Aguascalientes",22.370515,-102.312009,"22.3705155,-102.31200851052341",0,0,0,0,0,0,...,71,98,POINT (-102.312 22.371),NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Jesús María, Aguascalientes",21.934556,-102.468452,"21.934555500000002,-102.46845171651151",0,0,0,0,0,0,...,12,20,POINT (-102.468 21.935),NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"San Francisco de los Romo, Aguascalientes",22.074628,-102.270713,"22.0746282,-102.2707133",0,0,0,0,0,0,...,15,18,POINT (-102.271 22.075),NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Ensenada, Baja California",31.865889,-116.602983,"31.8658887,-116.602983",4,0,4,1,0,1,...,6,29,POINT (-116.603 31.866),NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,"Valladolid, Yucatán",20.690251,-88.201700,"20.6902505,-88.2016999",0,0,0,0,0,0,...,0,5,POINT (-88.202 20.690),NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,"Calera, Zacatecas",22.978480,-102.856223,"22.978479999999998,-102.85622288611614",0,0,0,0,0,0,...,16,42,POINT (-102.856 22.978),NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,"Jerez, Zacatecas",22.710593,-103.001473,"22.7105933,-103.00147264243049",0,0,0,0,0,0,...,0,1,POINT (-103.001 22.711),NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,"Trancoso, Zacatecas",22.735302,-102.366797,"22.7353025,-102.3667969",0,0,0,0,0,0,...,79,96,POINT (-102.367 22.735),NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Archivos para descarga

#### Pruebas: '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/2. datos_estadisticos/1. Detenciones INM/Total_De_Eventos/Eventos Municipal/3.1_mun_2023_pruebas.xlsx'

### Tabla de datos unida con los centroides (solo puntos, funciona muy bien para hacer heatmaps ) 389 municipios con datos y los poligonos de todos los municipios de municipios.

In [51]:
# Asegúrate de que 'datos_municipales' es el nombre de tu DataFrame
#3.1_mun_2023_centr.csv
datos_municipio.to_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/2. datos_estadisticos/1. Detenciones INM/Total_De_Eventos/Eventos Municipal/3.1_mun_2023_centr.csv', index=False)


### Tabla unida con 389 municipios con datos y los poligonos de todos los municipios de municipios

### Tabla unida centralmente datos y geodatos de solo los 389 municipios con datos